<h2>Lab 3: Least Squares</h2>

<b>Demo Date: </b> Oct. 13 <br>
<b>Due Date: </b> Oct. 15

In this lab you will implement two algorithms for solving an ill-conditioned least squares problem. We create an artificial and overdetermined least-squares problem by removing two columns of a $10 \times 10$ Hilbert matrix, a classic ill-conditioned matrix. As we remove the two columns from the matrix it is no longer a Hilbert matrix, but it creates an overdetermined system with a large condition number: $\approx 3,796,554,168$.

Implement an algorithm that you believe will compute the value of $x$ for the least squares problem $Ax \approx b$ as accurately as the problem allows. You will also implement an algorithm that you believe to produce inaccurate solutions due to the large condition number of the system. In this lab you do not need to worry about having a fast implementation, any non-vectorized implementation will be enough. 

Compare the two solutions in terms of the norm of their residual vector $Ax - b$. In the demo of your lab you should be able to explain your choices of algorithms and the results you obtained. Why did one method produce a solution with a smaller residual than the other? 

In [ ]:
import numpy as np
import copy
import random
from scipy.linalg import hilbert
from scipy.optimize import lsq_linear


# library method
def lsq_v4_library(A, b):
  x = lsq_linear(A, b)
  return x.x


# Normal equation
# In theory the system of normal equations gives the exact solution to
# a linear least square problem, but in practice this approach sometimes 
# yields disappointingly inaccurate results. There are two reasons for this behavior: 
# 1. Information can be lost in forming the cross-product matrix and right-hand-side vector.
# 2. The condition number of the cross-product matrix, which determines the sensitivity of 
# the solution to the normal equations, is the square of that of the original matrix A.
# There is a potential condition-squaring effect in the sensitive of least square solutions, 
# but this should be a significant factor only when the residual is large.
# Unfortunately, the normal equation can suffer a condition-squaring effect even when the fit is good 
# and the residual is small, making the computed solution more sensitive than warranted by the underlying least squares problem.
# In this sense, the normal equations method is unstable.
def lsq_v3_normal(A, b):
  b = b.reshape((len(b), 1))
  # At_A * x = At_b
  At = np.transpose(A)
  At_A = np.dot(At, A)
  At_b = np.dot(At, b)
  # We use LU factorization to solve this here
  L, U = lu_factor(copy.deepcopy(At_A))
  y = forward_substituion(L, At_b)
  x = back_substituion(U, y)     
  return x


def forward_substituion(L, b):
  # change the data type of b
  n = len(L)
  x = np.zeros((L.shape[1], b.shape[1]))
  for j in range(0, n):
    if L[j][j] == 0:
      break # singular matrix
    x[j] = b[j] / L[j][j]

    for i in range(j, n):
      b[i] = b[i] - L[i][j] * x[j]
  return x


def back_substituion(U, b):
  # change the data type of b
  n = len(U)
  x = np.zeros((U.shape[1], b.shape[1]))
  for j in range(n - 1, -1, -1):
    if U[j][j] == 0:
      break # singular matrix
    x[j] = b[j] / U[j][j]

    for i in range(0, j):
      b[i] = b[i] - U[i][j] * x[j]
  return x


def lu_factor(A):
  # Another way to do this part
  # change the datatype of A
  A = A + 0.0
  n = len(A)
  U = copy.deepcopy(A)
  L = np.eye(n, k = 0)
  # iterate from the first column to the column before the last column
  for k in range(0, n - 1):
    if A[k][k] == 0:
      break
    # create a len(A) * len(A) identity matrix
    # I = np.eye(len(A), k = 0, dtype = float)
    I = np.eye(n, k = 0)
    # create an e_k which is the transpose of the kth column of the identity matrix
    e_k = np.zeros(shape = (1, n))
    e_k[0][k] = 1
    # create a m which is the kth column of the matrix U
    m = copy.deepcopy(U[:, k])
    m = m.reshape(n, 1)
    # m_i = a_i / a_k, i = k + 1, ...
    m = np.divide(m, U[k][k])
    # set the values before m_k+1 equal to 0
    m[0:(k + 1), 0] = 0
    # M = I - m * e_k
    M = I - np.dot(m, e_k)
    # L = I + m * e_k
    l = I + np.dot(m, e_k)
    # M_n * ... * M_2 * M_1 * A = U
    U = np.dot(M, U)
    # L_1 * L_2 * ... * L_n = L
    L = np.dot(L, l)

  return L, U


# Modified Gram-Schmidt
# Modified Gram-Schmidt is equivalent mathematically, but superior numerically,
# to classical Gram-Schmidt. Unlike either of the column-oriented Gram-Schmidt procedures, 
# the roworiented modified Gram-Schmidt procedure permits the use of column pivoting to identify 
# a maximal linearly independent set of columns of A.
def lsq_v1_MGS(A, b):
  b = b.reshape((len(b), 1))
  m = A.shape[0]
  n = A.shape[1]
  # Create two matrix Q and R
  R = np.zeros((n, n))
  Q = np.zeros((m, n))
  for k in range(0, n):        # loop over columns
    R[k][k] = np.linalg.norm(A[:, k], ord = 2)
    if R[k][k] == 0:         # stop if linearly dependent
      break
    Q[:, k] = A[:, k] /  R[k][k]       
    for j in range(k + 1, n):        # substract from succeeding columns there components in current column
      Qk_t = np.transpose(Q[:, k])
      R[k][j] = np.dot(Qk_t, A[:, j])
      A[:, j] = A[:, j] - R[k][j] * Q[:, k]
  # c1 = Q^T * b
  Q1_t = np.transpose(Q)
  c1 = np.dot(Q1_t, b)
  x = back_substituion(R, c1)
  return x


# Classic Gram-Schmidt
# The classical Gram-Schmidt procedure is less than satisfactory when implemented 
# in finite-precision arithmetic, as orthogonality among the computed qk tends to be 
# lost due to rounding error. Moreover, the classical Gram-Schmidt procedure requires 
# separate storage for A and Q1 (as well as R) because the original ak is used in the 
# inner loop, and hence qx, which is updated in the inner loop, cannot overwrite it. 
def lsq_v5_CGS(A, b):
  b = b.reshape((len(b), 1))
  m = A.shape[0]
  n = A.shape[1]
  # Create two matrix Q and R
  R = np.zeros((n, n))
  Q = np.zeros((m, n))
  for k in range(0, n):    # loop over columns
    Q[:, k] = A[:, k]
    for j in range(0, k):         # substract from current column its components in preceding columns
      Qj_t = np.transpose(Q[:, j])
      R[j][k] = np.dot(Qj_t, A[:, k])
      Q[:, k] = Q[:, k] - R[j][k] * Q[:, j]
    R[k][k] = np.linalg.norm(Q[:, k], ord = 2)
    if R[k][k] == 0:      # stop if linearly dependent
      break
    Q[:, k] = Q[:, k] / R[k][k]    # normalize current column
  Q1_t = np.transpose(Q)
  # c1 = Q^T * b
  c1 = np.dot(Q1_t, b)
  x = back_substituion(R, c1)
  return x


# Modified Gram-Schmidt with augmented matrix 
# Even with the modified Gram-Schmidt procedure, cancellation can still 
# occur when components in one vector are subtracted from another, leading 
# to a significant loss of orthogonality among the columns of Q1 when A is 
# ill-conditioned, though the loss is much less severe than with classical Gram-Schmidt.
# For this reason, when using modified Gram-Schmidt to solve a linear least squares problem Ax = b, 
# it is not advisable to use the computed Q1 explicitly to compute the transformed right-hand 
# side c = Q1^T * b. Instead, it is better numerically to treat the right-hand-side vector b 
# as an (n + 1)-st column, using modified Gram-Schmidt to compute the reduced QR factorization of 
# the resulting m × (n + 1) augmented matrix.
def lsq_v6_augmented_matrix_MGS(A, b):
  A = A.reshape((len(A), len(A[0])))
  b = b.reshape((len(b), 1))
  new_A = np.zeros((len(A), len(A[0]) + 1), dtype=float)
  new_A[0:, 0:len(A[0])] = A
  new_A[0:, len(A[0]):] = b
  m = new_A.shape[0]
  n = new_A.shape[1]
  # Create two matrix Q and R
  R = np.zeros((n, n), dtype=float)
  Q = np.zeros((m, n), dtype=float)
  for k in range(0, n):
    R[k][k] = np.linalg.norm(new_A[0:, k], ord=2)
    if R[k][k] == 0:
      break
    Q[0:, k] = new_A[0:, k] /  R[k][k]
    for j in range(k + 1, n):
      Qk_t = np.transpose(Q[0:, k])
      R[k][j] = np.dot(Qk_t, new_A[0:, j])
      new_A[0:, j] = new_A[0:, j] - R[k][j] * Q[0:, k]
  new_R = np.zeros((n - 1, n - 1), dtype=float)
  new_R = R[0:n-1, 0:n-1]
  new_c1 = np.zeros((n - 1, 1), dtype=float)
  new_c1 = R[0:n-1, n-1]
  new_c1 = new_c1.reshape((n - 1, 1))
  x = back_substituion(new_R, new_c1)
  return x
  

# Householder
# Each Householder transformation must be applied to the remaining unreduced 
# portion of the matrix, but it will not affect the prior columns already reduced, 
# and hence the zeros are preserved through successive transformations.
# Moreover, unnecessary overflow or underflow can be avoided by appropriate scaling.
def lsq_v2_householder(A, b):
  b = b.reshape((len(b), 1))
  m = A.shape[0]
  n = A.shape[1]
  for k in range(0, n):      # loop over columns
    # Ak = -sign(Akk) * sqrt(Akk^2 + ... + Amk^2)
    alpha_k = (-1) * np.sign(A[k][k]) * np.linalg.norm(A[k:, k], ord=2)    # compute Householder vector for current col
    # Vk = [0, ..., 0, Akk, ..., Amk]^T - Akek
    Vk = np.zeros((m, 1))
    Vk = Vk.reshape((m, 1))
    t = np.zeros((m, 1))
    t = t.reshape((m, 1))
    e_k = np.zeros(shape = (m, 1))
    e_k = e_k.reshape((m, 1))
    e_k[k][0] = 1
    t[k:, 0] = A[k:, k]
    Vk = t - alpha_k * e_k
    V_k_t = np.transpose(Vk)
    # beta_k = Vk^T * Vk
    beta_k = np.dot(V_k_t, Vk)
    # if beta_k = 0 then
    if beta_k == 0:      # skip current column if it is already 0
      # continue with next k
      continue
    for j in range(k, n):    # apply transformation to remaining submatrix
      Aa = np.zeros((m, 1))
      Aa = A[0:, j]
      Aa = Aa.reshape((m, 1))
      gamma_j = np.dot(V_k_t, Aa)
      # Aj = Aj - (2 * gamma_j / beta_k) * Vk
      A[0:, j] = A[0:, j] - (2 * gamma_j[0][0] / beta_k[0][0]) * Vk[0:, 0]
    # we need to do some transformation to b to get Hb
    Bb = np.zeros((m, 1))
    Bb = b[0:, 0]
    Bb = Bb.reshape((m, 1))
    gamma_j = np.dot(V_k_t, Bb)
    b[0:, 0] = b[0:, 0] - (2 * gamma_j[0][0] / beta_k[0][0]) * Vk[0:, 0]
    
  R = np.zeros((n, n))
  R[0:, 0:] = A[0:n, 0:]
  c1 = np.zeros((n, 1))  
  c1 = c1.reshape(n, 1)
  c1[0:, 0] = b[0:n, 0]
  x = back_substituion(R, c1)
  return x



b = np.ones(10)
A = hilbert(10)
A = np.delete(A, 5, 1)
A = np.delete(A, 4, 1)

print('Condition number of A: ', np.linalg.norm(A) * np.linalg.norm(np.linalg.pinv(A)))
print("\n")
# Use this test instance while developing the algorithms. 
# This instance is much easier to debug than the ill-conditioned problem above. 
A_test = np.array([[1, 2, 2], [4, 4, 2], [4, 6, 4]], dtype=float)
b_test = np.array([3, 6, 10], dtype=float)

#A = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1], [-1, 1, 0], [-1, 0, 1], [0, -1, 1]], dtype=float)
#b = np.array([1237, 1941, 2417, 711, 1177, 475], dtype=float)

#A = np.array([[1, 2, 2], [4, 4, 2], [4, 6, 4]], dtype=float)
#b = np.array([3, 6, 10], dtype=float)


x3 = lsq_v3_normal(copy.deepcopy(A), copy.deepcopy(b))
#print(x3)
print('Normal equation residual norm: ', np.linalg.norm(A.dot(x3) - b.reshape((len(b), 1)), ord = 2))

x5 = lsq_v5_CGS(copy.deepcopy(A), copy.deepcopy(b))
#print(x5)
print('Classic Gram-Schmidt residual norm: ', np.linalg.norm(A.dot(x5) - b.reshape((len(b), 1)), ord = 2))

x1 = lsq_v1_MGS(copy.deepcopy(A), copy.deepcopy(b))
#print(x1)
print('Modified Gram-Schmidt residual norm: ', np.linalg.norm(A.dot(x1) - b.reshape((len(b), 1)), ord = 2))

x6 = lsq_v6_augmented_matrix_MGS(copy.deepcopy(A), copy.deepcopy(b))
#print(x6)
print('Modified Gram-Schmidt with augmented matrix residual norm: ', np.linalg.norm(A.dot(x6) - b.reshape((len(b), 1)), ord = 2))

x2 = lsq_v2_householder(copy.deepcopy(A), copy.deepcopy(b))
#print(x2)
print('Householder residual norm: ', np.linalg.norm(A.dot(x2) - b.reshape((len(b), 1)), ord = 2))

x4 = lsq_v4_library(copy.deepcopy(A), copy.deepcopy(b))
#print(x4)
print('Library method residual norm: ', np.linalg.norm(A.dot(x4) - b, ord = 2))


Condition number of A:  3796554168.541331


Normal equation residual norm:  0.0001375497387011829
Classic Gram-Schmidt residual norm:  0.0010355321472345004
Modified Gram-Schmidt residual norm:  1.3094886573253993e-05
Modified Gram-Schmidt with augmented matrix residual norm:  1.3094846972702966e-05
Householder residual norm:  1.3094845961453556e-05
Library method residual norm:  1.3094846687100912e-05
